In [110]:
# Enables figures loading outside of browser.
# If not run, figures will load inline.
%matplotlib

import os
import math
import pandas as pd
import numpy as np
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import matplotlib.ticker
import datetime
import collections
import itertools

from scipy import stats

# Depends on: pip install sklearn
from sklearn.model_selection import train_test_split

# Some matplotlib features are version dependent.
assert(matplotlib.__version__ >= '2.1.2')

# Depends on: pip install --upgrade google-cloud-bigquery
from google.cloud import bigquery

def run_query(query, project='mlab-sandbox'):
    #print query
    client = bigquery.Client(project=project)
    job = client.query(query)

    results = collections.defaultdict(list)
    for row in job.result(timeout=300):
        for key in row.keys():
            results[key].append(row.get(key))

    return pd.DataFrame(results)

def unlog(x, pos):
    """Formats the x axis for histograms taken on the log of values."""
    v = math.pow(10, x)
    frac, whole = math.modf(v)
    if frac > 0:
        return '%.1f' % v
    else:
        return '%d' % whole

logFormatter = matplotlib.ticker.FuncFormatter(unlog)

Using matplotlib backend: MacOSX


In [156]:
def plot_scatter(
    df, xname, yname,
    fig_by='', axes_by='', group_by='',
    figsize=(6,8), axes=(1,1),
    xlabel='', ylabel='',
    xlim=(), ylim=(),
    fx=list, fy=list,
    ylog=False, title='', legend={}):
    """
    Args:
        xname: str, name of column to use as x-axis.
        yname: str, name of column to use as y-axis.
        fig_by: str, name of column to split data into multiple figures.
        axes_by: str, name of column to arrange into a single panel.
        group_by: str, name of column to plot common split_by and group_by columns.
        figsize: (int, int), dimensions of figure.
        axes: (int, int), arrangement of axes within figure.
        xlabel: str, 
        ylabel: str, 
        fx: func,
        fy: func,
        xlim: (xmin, xmax),
        ylim: (ymin, ymax),
        ylog: bool,
        title: str,
        legend: **legend_args,
    """
    for fname in sorted(set(['default'] if not fig_by else df[fig_by])):
        fig = plt.figure(figsize=figsize)
        ax = fig.subplots(axes[0], axes[1], squeeze=False)
        ax_index = list(itertools.product(range(axes[0]), range(axes[1])))
        
        #print fig_by, fname
        df_fig = df if fname == 'default' else df[df[fig_by] == fname]
        for p, a in enumerate(sorted(set(['default'] if not axes_by else df_fig[axes_by]))):
            #print axes_by, a
            df_axes = df_fig if a == 'default' else df_fig[df_fig[axes_by] == a]

            i, j = ax_index[p]
            for g in sorted(set(['default'] if not group_by else df_axes[group_by])):
                df_g = df_axes if g == 'default' else df_axes[df_axes[group_by] == g]

                x = fx(df_g[xname])
                y = fy(df_g[yname])

                ax[i][j].scatter(x, y, s=1, label=g)

            if i != len(ax)-1:
                ax[i][j].set_xticklabels([])

            if ylabel:
                print a
                ax[i][j].set_ylabel(ylabel.format(a.upper()))
            if xlabel:
                ax[i][j].set_xlabel(xlabel)

            if xlim:
                ax[i][j].set_xlim(xlim)
            if ylim:
                ax[i][j].set_ylim(ylim)
            ax[i][j].tick_params(axis='x', labelrotation=-90)
            ax[i][j].grid(color='#dddddd')
            ax[i][j].legend(fontsize='x-small', **legend)
            if ylog:
                ax[i][j].semilogy()

        fig.suptitle(title.format(fname))
        fig.tight_layout(rect=[0, 0.03, 1, 0.95])

# UPLINK UTILIZATION OVER TIME

In [150]:
df_disco_pct = run_query("""
#standardSQL
SELECT

  REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3})[0-9]{2}.*') AS metro,
  REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3}[0-9]{2}).*') AS site,
  FORMAT_TIMESTAMP("%Y-%m-%d", TIMESTAMP_TRUNC(sts, DAY)) AS day,
  UNIX_SECONDS(TIMESTAMP_TRUNC(sts, DAY)) AS ts,  
  APPROX_QUANTILES(value, 101)[ORDINAL(50)] as bytes_50th,
  APPROX_QUANTILES(value, 101)[ORDINAL(90)] as bytes_90th

FROM (
  SELECT
    metric,
    REGEXP_EXTRACT(hostname, r'(mlab1.[a-z]{3}[0-9]{2}).*') AS hostname,
    sample.timestamp AS sts,
    sample.value AS value
  FROM
    `measurement-lab.base_tables.switch*`,
    UNNEST(sample) AS sample
  WHERE
    metric LIKE 'switch.octets.uplink.tx'
    AND REGEXP_CONTAINS(hostname, r"mlab1.(dfw|lga|nuq)\d\d")
  GROUP BY
    hostname, metric, sts, value
)
WHERE
  hostname IS NOT NULL
GROUP BY
  hostname, day, ts
ORDER BY
  hostname, day, ts
""")

In [151]:
plot_scatter(
    df_disco_pct, 'ts', 'bytes_50th', axes_by='metro', group_by='site', axes=(3, 1),
    title='Daily Median Uplink Utilization',
    ylabel="Median Uplink {0}",
    xlim=(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01")),
    ylim=(1e4, 1e9),
    fx=lambda l: [pd.to_datetime(t, unit='s') for t in l],
    legend={'loc':3, 'ncol':7, 'columnspacing':1},
    ylog=True)

In [152]:
#sites = [
#    ['dfw'],
#    ['lga'],
#    ['nuq'],
#]
#
#axes = [
#    [None],
#    [None],
#    [None],
#]
#
#fig = plt.figure(figsize=(6, 8))
#for i, site_row in enumerate(sites):
#    for j, site in enumerate(site_row):
#        axes[i][j] = plt.subplot2grid((3, 1), (i, j))
#        axes[i][j].set_ylabel('Median Uplink ' + site.upper())##
#
#        if i != len(sites)-1:
#            axes[i][j].set_xticklabels([])
#        for h in set(df_disco_pct['hostname']):
#            if ('mlab1.' + site) in h:
#                ds = df_disco_pct[ (df_disco_pct['hostname'] == h) & (df_disco_pct['bytes_50th'] > 1e5) ]
#                d = [pd.to_datetime(t, unit='s') for t in ds['ts']]
#                axes[i][j].scatter(d, ds['bytes_50th'], s=1, label=h[6:11])
#                
#        axes[i][j].set_ylim(1e4, 1e9)
#        axes[i][j].set_xlim(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01"))
#        axes[i][j].tick_params(axis='x', labelrotation=-90)
#        axes[i][j].grid(color='#dddddd')
#        axes[i][j].legend(loc=3, ncol=7, fontsize='x-small', columnspacing=1)
#        axes[i][j].semilogy()
#
#fig.suptitle('Daily Median Uplink Utilization')
#fig.tight_layout(rect=[0, 0.03, 1, 0.95])

# Packets Over Time

In [158]:
df_disco_packets = run_query("""
#standardSQL

WITH measurementlab_switch_dedup AS (

  SELECT
    metric,
    REGEXP_EXTRACT(hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS hostname,
    sample.timestamp AS sts,
    sample.value AS value
    
  FROM
    `measurement-lab.base_tables.switch*`,
    UNNEST(sample) AS sample
    
  WHERE
    metric LIKE 'switch.unicast.uplink.tx'
    AND REGEXP_CONTAINS(hostname, r"mlab1.(dfw|lga|nuq)\d\d")
    
  GROUP BY
    hostname, metric, sts, value
)

SELECT
  REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3})[0-9]{2}.*') AS metro,
  REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3}[0-9]{2}).*') AS site,
  hostname,
  FORMAT_TIMESTAMP("%Y-%m-%d", TIMESTAMP_TRUNC(sts, DAY)) AS day,
  UNIX_SECONDS(TIMESTAMP_TRUNC(sts, DAY)) AS ts,
  SUM(value) AS total
  
FROM
  measurementlab_switch_dedup
  
WHERE
  hostname IS NOT NULL
  
GROUP BY
  hostname, day, ts
  
ORDER BY
  hostname, day, ts
""")

In [160]:
plot_scatter(
    df_disco_packets, 'ts', 'total', axes_by='metro', group_by='site', axes=(3, 1),
    title='Daily Packets',
    ylabel="Total Packets {0}",
    xlim=(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01")),
    ylim=(1e7, 1e10),
    fx=lambda l: [pd.to_datetime(t, unit='s') for t in l],
    legend={'loc':3, 'ncol':7, 'columnspacing':1},
    ylog=True)

dfw
lga
nuq


In [161]:
#sites = [
#    ['dfw'],
#    ['lga'],
#    ['nuq'],
#]
#
#axes = [
#    [None],
#    [None],
#    [None],
#]
#
#fig = plt.figure(figsize=(6, 8))
#
#for i, site_row in enumerate(sites):
#    for j, site in enumerate(site_row):
#        axes[i][j] = plt.subplot2grid((3, 1), (i, j))
#        axes[i][j].set_ylabel('Total Packets ' + site.upper())
#        if i != len(sites)-1:
#            axes[i][j].set_xticklabels([])
#
#        for h in set(df_disco_packets['hostname']):
#            if ('mlab1.' + site) in h:
#                ds = df_disco_packets[ (df_disco_packets['hostname'] == h) & (df_disco_packets['total'] > 100) ]
#                d = [pd.to_datetime(t, unit='s') for t in ds['ts']]
#                axes[i][j].scatter(d, ds['total'], s=1, label=h[6:11])
#
#        axes[i][j].set_ylim(1e7, 1e10)
#        axes[i][j].set_xlim(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01"))
#        axes[i][j].tick_params(axis='x', labelrotation=-90)
#        axes[i][j].grid(color='#dddddd')
#        axes[i][j].legend(loc=3, ncol=7, fontsize='x-small')
#        axes[i][j].semilogy()
#        
#fig.suptitle('Daily Packets')
#fig.tight_layout(rect=[0, 0.03, 1, 0.95])

# DISCARDS OVER TIME

In [164]:
df_disco = run_query("""
#standardSQL

WITH measurementlab_switch_dedup AS (

  SELECT
    metric,
    REGEXP_EXTRACT(hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS hostname,
    sample.timestamp AS sts,
    sample.value AS value
    
  FROM
    `measurement-lab.base_tables.switch*`,
    UNNEST(sample) AS sample
    
  WHERE
    metric LIKE 'switch.discards.uplink.tx'
    AND REGEXP_CONTAINS(hostname, r"mlab1.(dfw|lga|nuq)\d\d")
    
  GROUP BY
    hostname, metric, sts, value
)

SELECT
  REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3})[0-9]{2}.*') AS metro,
  REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3}[0-9]{2}).*') AS site,
  hostname,
  FORMAT_TIMESTAMP("%Y-%m-%d", TIMESTAMP_TRUNC(sts, DAY)) AS day,
  UNIX_SECONDS(TIMESTAMP_TRUNC(sts, DAY)) AS ts,
  SUM(value) AS total_discards
  
FROM
  measurementlab_switch_dedup
  
WHERE
  hostname IS NOT NULL
  
GROUP BY
  hostname, day, ts
  
ORDER BY
  hostname, day, ts
""")

In [162]:
sites = [
    ['dfw'],
    ['lga'],
    ['nuq'],
]

axes = [
    [None],
    [None],
    [None],
]

fig = plt.figure(figsize=(6, 8))

for i, site_row in enumerate(sites):
    for j, site in enumerate(site_row):
        axes[i][j] = plt.subplot2grid((3, 1), (i, j))
        axes[i][j].set_ylabel('Total Discards ' + site.upper())
        if i != len(sites)-1:
            axes[i][j].set_xticklabels([])
        #else:
        #    axes[i][j].set_xlabel('Date')
            
        for h in set(df_disco['hostname']):
            if ('mlab1.' + site) in h:
                ds = df_disco[ (df_disco['hostname'] == h) & (df_disco['total_discards'] > 100) ]
                d = [pd.to_datetime(t, unit='s') for t in ds['ts']]
                axes[i][j].scatter(d, ds['total_discards'], s=1, label=h[6:11])

        axes[i][j].set_ylim(100, 1000000)
        axes[i][j].set_xlim(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01"))
        axes[i][j].tick_params(axis='x', labelrotation=-90)
        axes[i][j].grid(color='#dddddd')
        axes[i][j].legend(loc=2, fontsize='x-small')
        axes[i][j].semilogy()
        
fig.suptitle('Daily Packet Discards')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])


# Daily DISCO discard ratios

In [168]:
df_disco_ratio = run_query("""
WITH measurementlab_switch_dedup AS (
  SELECT
    metric,
    REGEXP_EXTRACT(hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS hostname,
    sample.timestamp AS sts,
    sample.value AS value
  FROM
    `measurement-lab.base_tables.switch*`,
    UNNEST(sample) AS sample
  WHERE
    (metric LIKE 'switch.discards.uplink.tx' OR metric LIKE 'switch.unicast.uplink.tx')
    AND (hostname LIKE '%mlab1.lga%' OR hostname LIKE '%mlab1.dfw%' OR hostname LIKE '%mlab1.nuq%')
  GROUP BY
    hostname, metric, sts, value
)

SELECT
  REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3})[0-9]{2}.*') AS metro,
  REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3}[0-9]{2}).*') AS site,
  hostname,
  day,
  ts,
  IF(total > 0, discards / total, 0) as ratio
FROM (
SELECT
  hostname,
  FORMAT_TIMESTAMP("%Y-%m-%d", TIMESTAMP_TRUNC(sts, DAY)) AS day,
  UNIX_SECONDS(TIMESTAMP_TRUNC(sts, DAY)) AS ts,
  SUM(IF(metric = "switch.discards.uplink.tx", value, 0)) AS discards,
  SUM(IF(metric = "switch.unicast.uplink.tx", value, 0)) AS total
FROM
  measurementlab_switch_dedup
WHERE
  hostname IS NOT NULL
GROUP BY
  hostname, day, ts
HAVING
  discards < total
ORDER BY
  hostname, day, ts
)
GROUP BY
  hostname, day, ts, ratio
HAVING
  ratio < 0.01
ORDER BY
  hostname, day, ts
""")

In [211]:
plot_scatter(
    df_disco_ratio, 'ts', 'ratio', axes_by='metro', group_by='site', axes=(3, 1),
    title='Daily Packet Loss Ratio',
    ylabel="Discard Ratio {0}",
    xlim=(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01")),
    ylim=(1e-6, 1e-1),
    fx=lambda l: [pd.to_datetime(t, unit='s') for t in l],
    legend={'loc':2},
    ylog=True)

dfw
lga
nuq


In [165]:
#sites = [
#    ['dfw'],
#    ['lga'],
#    ['nuq'],
#]
#
#axes = [
#    [None],
#    [None],
#    [None],
#]
#
#fig = plt.figure(figsize=(6, 8))
#
#for i, site_row in enumerate(sites):
#    for j, site in enumerate(site_row):
#        axes[i][j] = plt.subplot2grid((3, 1), (i, j))
#        axes[i][j].set_ylabel('Discard Ratio ' + site.upper())
#        if i != len(sites)-1:
#            axes[i][j].set_xticklabels([])
#            
#        for h in set(df_disco_ratio['hostname']):
#            if ('mlab1.' + site) in h:
#                ds = df_disco_ratio[ (df_disco_ratio['hostname'] == h) ]
#                d = [pd.to_datetime(t, unit='s') for t in ds['ts']]
#                axes[i][j].scatter(d, ds['ratio'], s=1, label=h[6:11])
#
#        axes[i][j].set_ylim(1e-6, 1e-2)
#        axes[i][j].set_xlim(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01"))
#        axes[i][j].tick_params(axis='x', labelrotation=-90)
#        axes[i][j].grid(color='#dddddd')
#        axes[i][j].legend(loc=2, fontsize='x-small')
#        axes[i][j].semilogy()
#
#fig.suptitle('Daily Packet Loss Ratios')
#fig.tight_layout(rect=[0, 0.03, 1, 0.95])

# NDT Median Download Rates

In [10]:
df_ndt_median = run_query("""
WITH mlab_ndt AS (
  SELECT
    connection_spec.server_hostname as server_hostname,
    log_time,
    (8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd))) AS download_mbps

  FROM
  
    `measurement-lab.base_tables.ndt*`
  WHERE

        REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw|lga|nuq)\d\d")
    AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
    AND connection_spec.data_direction = 1
    AND web100_log_entry.connection_spec.remote_ip != "45.56.98.222"
    AND web100_log_entry.connection_spec.remote_ip != "2600:3c03::f03c:91ff:fe33:819"
    AND web100_log_entry.connection_spec.remote_ip != "35.225.75.192"
    AND web100_log_entry.connection_spec.remote_ip != "35.192.37.249"
    AND web100_log_entry.connection_spec.remote_ip != "35.193.254.117"
    
  
  GROUP BY
    server_hostname,
    log_time,
    web100_log_entry.connection_spec.remote_ip,
    web100_log_entry.connection_spec.local_ip,
    web100_log_entry.connection_spec.remote_port,
    web100_log_entry.connection_spec.local_port,
    download_mbps
)
    
SELECT
  TIMESTAMP_TRUNC(log_time, DAY) as day,
  REGEXP_EXTRACT(server_hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS hostname,
  APPROX_QUANTILES(download_mbps, 101)[ORDINAL(50)] as download_mbps
  -- AVG(download_mbps) as download_mbps
FROM
  mlab_ndt

GROUP BY
  day,
  hostname

ORDER BY
  day
""")

In [170]:
sites = [
    ['dfw'],
    ['lga'],
    ['nuq'],
]

axes = [
    [None],
    [None],
    [None],
]

print len(df_ndt_median)

fig = plt.figure(figsize=(6, 8))

for i, site_row in enumerate(sites):
    for j, site in enumerate(site_row):
        axes[i][j] = plt.subplot2grid((3, 1), (i, j))
        axes[i][j].set_ylabel('Mbps ' + site.upper())
        if i != len(sites)-1:
            axes[i][j].set_xticklabels([])
            
        for h in set(df_ndt_median['hostname']):
            if ('mlab1.' + site) in h:
                ds = df_ndt_median[ (df_ndt_median['hostname'] == h) ]
                d = [pd.to_datetime(t) for t in ds['day']]
                axes[i][j].scatter(d, ds['download_mbps'], s=1, label=h[6:11])

        #axes[i][j].set_ylim(100, 1000000)
        axes[i][j].set_ylim(1, 100)
        axes[i][j].set_xlim(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01"))
        axes[i][j].tick_params(axis='x', labelrotation=-90)
        axes[i][j].grid(color='#dddddd')
        axes[i][j].legend(loc=2, fontsize='x-small')
        axes[i][j].semilogy()
        
fig.suptitle('Median NDT Download Rates')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

8032


# NDT Segs Retrans

In [12]:
# NOT ENOUGH HISTORICAL NDT DATA TO GET FULL TIMELINE.

df_ndt_retrans = run_query("""
#standardSQL

SELECT
  TIMESTAMP_TRUNC(log_time, DAY) AS day,
  REGEXP_EXTRACT(connection_spec.server_hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS hostname,
  SAFE_DIVIDE(SUM(web100_log_entry.snap.SegsRetrans), SUM(web100_log_entry.snap.SegsOut)) AS median_ratio
FROM
  `measurement-lab.base_tables.ndt*`
WHERE
  web100_log_entry.snap.HCThruOctetsAcked >= 1000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
  AND REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw|lga|nuq|ord|atl|lax)\d\d")
  AND connection_spec.data_direction = 1
  AND log_time >= TIMESTAMP("2016-06-01")
GROUP BY
  day,
  hostname
HAVING
  median_ratio is not NULL
ORDER BY
  day
""")

In [195]:
df_ndt_retrans = run_query("""
WITH mlab_ndt AS (
  SELECT
    connection_spec.server_hostname as hostname,
    web100_log_entry.connection_spec.remote_ip as remote_ip,
    log_time,
    web100_log_entry.snap.SegsRetrans as SegsRetrans,
    web100_log_entry.snap.SegsOut as SegsOut

  FROM
    `measurement-lab.base_tables.ndt*`

  WHERE
        REGEXP_CONTAINS(connection_spec.server_hostname, r"(lga|dfw|nuq)\d\d")
    AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
    AND connection_spec.data_direction = 1
    AND web100_log_entry.connection_spec.remote_ip != "45.56.98.222"
    AND web100_log_entry.connection_spec.remote_ip != "2600:3c03::f03c:91ff:fe33:819"
    AND web100_log_entry.connection_spec.remote_ip != "35.225.75.192"
    AND web100_log_entry.connection_spec.remote_ip != "35.192.37.249"
    AND web100_log_entry.connection_spec.remote_ip != "35.193.254.117"
    AND log_time >= TIMESTAMP("2016-06-01")
  
  GROUP BY
    connection_spec.server_hostname,
    log_time,
    web100_log_entry.connection_spec.remote_ip,
    web100_log_entry.connection_spec.local_ip,
    web100_log_entry.connection_spec.remote_port,
    web100_log_entry.connection_spec.local_port,
    SegsRetrans,
    SegsOut
)
    
SELECT
  REGEXP_EXTRACT(hostname, r"([a-z]{3})[0-9]{2}") as metro,
  REGEXP_EXTRACT(hostname, r"([a-z]{3}[0-9]{2})") as site,
  day,
  APPROX_QUANTILES(ratio, 101)[ORDINAL(50)] AS median_ratio,
  count(*) as count
FROM
(
  SELECT
    hostname,
    TIMESTAMP_TRUNC(log_time, DAY) as day,
    MAX(SAFE_DIVIDE(SegsRetrans, SegsOut)) as ratio

  FROM
    mlab_ndt

  GROUP BY
    hostname,
    day,
    remote_ip
)

GROUP BY
  hostname, day

ORDER BY
  day
""")

In [209]:
#sites = [
#    ['dfw'],
#    ['lga'],
#    ['nuq'],
#]
#
#axes = [
#    [None],
#    [None],
#    [None],
#    [None],
#    [None],
#    [None],#
#
#]
#
#print len(df_ndt_retrans)
#
#fig = plt.figure(figsize=(6, 8))
#
#for i, site_row in enumerate(sites):
#    for j, site in enumerate(site_row):
#        axes[i][j] = plt.subplot2grid((3, 1), (i, j))
#        axes[i][j].set_ylabel('Retransmission ' + site.upper())
#        if i != len(sites)-1:
#            axes[i][j].set_xticklabels([])#
#
#        for s in sorted(set(df_ndt_retrans['site'])):
#            if site in s:
#                ds = df_ndt_retrans[ (df_ndt_retrans['site'] == s) ]
#                d = [pd.to_datetime(t) for t in ds['day']]
#                axes[i][j].scatter(d, ds['median_ratio'], s=1, label=s)
#
#        #axes[i][j].set_ylim(100, 1000000)
#        #axes[i][j].set_ylim(1e-6, 1e-2)
#        axes[i][j].set_ylim(0, 0.005)
#        axes[i][j].set_xlim(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01"))
#        axes[i][j].tick_params(axis='x', labelrotation=-90)
#        axes[i][j].grid(color='#dddddd')
#        axes[i][j].legend(loc=2, fontsize='x-small')
#        #axes[i][j].semilogy()
#        
#fig.suptitle('Median NDT SegsRetran/SegsOut Ratio')
#fig.tight_layout(rect=[0, 0.03, 1, 0.95])

In [210]:
plot_scatter(
    df_ndt_retrans, 'day', 'median_ratio', axes_by='metro', group_by='site', axes=(3, 1),
    title='Median NDT SegsRetran/SegsOut Ratio Rates',
    ylabel="Retransmission {0}",
    xlim=(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01")),
    ylim=(1e-6, 1e-1),
    fx=lambda l: [pd.to_datetime(t) for t in l],
    legend={'loc':2},
    ylog=True)

dfw
lga
nuq


In [80]:
def box(x, y, text):
    plt.text(x, y, text,
        bbox=dict(boxstyle="round",
              ec=(.5, 0.5, 1., 0.25),
              fc=(.5, 0.8, 1., 0.25),
        )
    )

In [95]:
## COMBINED SegsRetrans & Switch Discards

sites = [
    ['dfw'],
    ['lga'],
    ['nuq'],
]

axes = [
    [None],
    [None],
    [None],
    [None],
    [None],
    [None],
]

print len(df_ndt_retrans)

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

fig = plt.figure(figsize=(6, 8))

for i, site_row in enumerate(sites):
    for j, site in enumerate(site_row):
        axes[i][j] = plt.subplot2grid((3, 1), (i, j))
        axes[i][j].set_ylabel('Ratio ' + site.upper())
        if i != len(sites)-1:
            axes[i][j].set_xticklabels([])

        c = 0
        for s in sorted(set(df_ndt_retrans['site'])):
            if site in s:
                ds = df_ndt_retrans[ (df_ndt_retrans['site'] == s) ]
                d = [pd.to_datetime(t) for t in ds['day']]
                axes[i][j].scatter(d, ds['median_ratio'], s=1, label=s, c=colors[c])
                c += 1

        #axes[i][j].set_ylim(100, 1000000)
        axes[i][j].set_ylim(1e-6, 1e-1)
        #axes[i][j].set_ylim(0, 0.005)
        axes[i][j].set_xlim(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01"))
        axes[i][j].tick_params(axis='x', labelrotation=-90)
        axes[i][j].grid(color='#dddddd')
        axes[i][j].legend(loc=2, fontsize='x-small')
        axes[i][j].semilogy()

for i, site_row in enumerate(sites):
    for j, site in enumerate(site_row):       

        if i != len(sites)-1:
            axes[i][j].set_xticklabels([])
        c = 0
        for h in set(df_disco_ratio['hostname']):
            if ('mlab1.' + site) in h:
                ds = df_disco_ratio[ (df_disco_ratio['hostname'] == h) ]
                d = [pd.to_datetime(t, unit='s') for t in ds['ts']]
                axes[i][j].scatter(d, ds['ratio'], s=1, label=h[6:11], c=colors[c])
                c += 1

box(pd.to_datetime("2016-10-30"), 5e-3, u"Segs Retransmit ↘")
box(pd.to_datetime("2016-10-30"), 9e-6, u"Switch Discards ↗")
                
fig.suptitle('Retrans & Switch Discard Rates')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

23105


# One Week Performance Distributions -- Before & After

In [176]:
df_disco_dist = run_query("""
SELECT
    metric,
    REGEXP_EXTRACT(hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS name,
    CASE
        WHEN TIMESTAMP_TRUNC(sample.timestamp, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25") THEN 'before-2w'
        WHEN TIMESTAMP_TRUNC(sample.timestamp, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25") THEN 'after-2w'
        ELSE 'what'
    END as period,
    TIMESTAMP_TRUNC(sample.timestamp, DAY) AS day,
    UNIX_SECONDS(sample.timestamp) AS sts,
    8 * sample.value / 10 AS bps
  FROM
    `measurement-lab.base_tables.switch*`,
    UNNEST(sample) AS sample
  WHERE
       metric LIKE 'switch.octets.uplink.tx'
    AND (
        hostname LIKE '%mlab1.lga03%' OR hostname LIKE '%mlab1.dfw02%'
    )
    AND ( -- One week - Sunday to Saturday.
          TIMESTAMP_TRUNC(sample.timestamp, DAY) BETWEEN TIMESTAMP("2018-02-11") and TIMESTAMP("2018-02-25")
       OR TIMESTAMP_TRUNC(sample.timestamp, DAY) BETWEEN TIMESTAMP("2018-03-11") and TIMESTAMP("2018-03-25")
    )
  GROUP BY
    hostname, metric, sample.timestamp, bps
  ORDER BY
    hostname, sts
""")

In [177]:
print df_disco_dist.keys()
print len(df_disco_dist)



for h in set(df_disco_dist['name']):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))
    for day in sorted(set(df_disco_dist['period'])):
        ds = df_disco_dist[ (df_disco_dist['name'] == h) & (df_disco_dist['period'] == day) ]
        r = [math.log10(x) for x in ds['bps']]
            
        axes.hist(r, # len(ds['bps']),
                      int(1.5 * math.sqrt(len(ds['bps']))),
                      histtype='step',
                      density=1, # cumulative=True,
                      label='cdf-'+h[6:11] + '-' + str(day), ls='-')

        axes.set_title(h[6:11])
        #axes.tick_params(axis='x', labelrotation=90)
    axes.grid(color='#dddddd')
    axes.legend(loc=2, fontsize='x-small')
    axes.xaxis.set_major_formatter(logFormatter)
    fig.subplots_adjust(hspace=0.3, wspace=0.4)
    fig.suptitle('Uplink Utilization Distribution')

Index([u'bps', u'day', u'metric', u'name', u'period', u'sts'], dtype='object')
517644


# One Day Performance Distributions -- before & after

Harder to notice changes wrt before  after.

In [18]:
df_disco_dist4 = run_query("""
SELECT
    metric,
    REGEXP_EXTRACT(hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS name,
    TIMESTAMP_TRUNC(sample.timestamp, DAY) AS day,
    UNIX_SECONDS(sample.timestamp) AS sts,
    8 * sample.value / 10 AS bps
  FROM
    `measurement-lab.base_tables.switch*`,
    UNNEST(sample) AS sample
  WHERE
       metric LIKE 'switch.octets.uplink.tx'
    AND (
        hostname LIKE '%mlab1.lga03%' OR hostname LIKE '%mlab1.dfw02%'
    )
    AND (
        TIMESTAMP_TRUNC(sample.timestamp, DAY) = TIMESTAMP("2018-02-18")
     OR TIMESTAMP_TRUNC(sample.timestamp, DAY) = TIMESTAMP("2018-02-25")
     OR TIMESTAMP_TRUNC(sample.timestamp, DAY) = TIMESTAMP("2018-03-04")
     OR TIMESTAMP_TRUNC(sample.timestamp, DAY) = TIMESTAMP("2018-03-11")
--     OR TIMESTAMP_TRUNC(sample.timestamp, DAY) = TIMESTAMP("2018-03-04")
--     OR TIMESTAMP_TRUNC(sample.timestamp, DAY) = TIMESTAMP("2018-03-05")
--     OR TIMESTAMP_TRUNC(sample.timestamp, DAY) = TIMESTAMP("2018-03-10")
--     OR TIMESTAMP_TRUNC(sample.timestamp, DAY) = TIMESTAMP("2018-03-01")
--     OR TIMESTAMP_TRUNC(sample.timestamp, DAY) = TIMESTAMP("2018-03-01")
    )
  GROUP BY
    hostname, metric, sample.timestamp, bps
  ORDER BY
    hostname, sts
""")

In [97]:
print df_disco_dist4.keys()
print len(df_disco_dist4)



for h in set(df_disco_dist4['name']):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))
    for day in sorted(set(df_disco_dist4['day'])):
        ds = df_disco_dist4[ (df_disco_dist4['name'] == h) & (df_disco_dist4['day'] == day) ]
        r = [math.log10(x) for x in ds['bps']]
            
        axes.hist(r, # len(ds['bps']),
                      int(1.5 * math.sqrt(len(ds['bps']))),
                      histtype='step',
                      density=1, # cumulative=True,
                      label='cdf-'+h[6:11] + '-' + str(day), ls='-')

        axes.set_title(h[6:11])
        #axes.tick_params(axis='x', labelrotation=90)
    axes.grid(color='#dddddd')
    axes.legend(loc=2, fontsize='x-small')
    axes.xaxis.set_major_formatter(logFormatter)
    fig.subplots_adjust(hspace=0.3, wspace=0.4)
    fig.suptitle('Uplink Utilization Distribution')

Index([u'bps', u'day', u'metric', u'name', u'sts'], dtype='object')
69120


# NDT test distributions - Before & After

In [20]:
if False:
    df_ndt_dist = run_query("""
-- ALL NDT tests before and after the flow-control configuration change.
   
SELECT
    REGEXP_EXTRACT(connection_spec.server_hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS name,
    CASE
     WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") and TIMESTAMP("2018-02-18")
       THEN 'before-2w'
     WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-18") and TIMESTAMP("2018-02-25")
       THEN 'before-1w'
     WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-04") and TIMESTAMP("2018-03-11")
       THEN 'after-1w'
     WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") and TIMESTAMP("2018-03-18")
       THEN 'after-2w'
     ELSE 'what'
    END as period,

    web100_log_entry.snap.StartTimeStamp AS ts,
    8 * (web100_log_entry.snap.HCThruOctetsAcked /
      (web100_log_entry.snap.SndLimTimeRwin +
       web100_log_entry.snap.SndLimTimeCwnd +
       web100_log_entry.snap.SndLimTimeSnd)) as download_mbps   
FROM
   `measurement-lab.base_tables.ndt*`
   
WHERE       
    ( -- One week - Sunday to Saturday.
        TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") and TIMESTAMP("2018-02-18")
     OR TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-18") and TIMESTAMP("2018-02-25")
     OR TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-04") and TIMESTAMP("2018-03-11")
     OR TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") and TIMESTAMP("2018-03-18"))
  AND REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw02|lga03)")
  AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000
  AND (web100_log_entry.snap.SndLimTimeRwin +                                   
    web100_log_entry.snap.SndLimTimeCwnd +                                      
    web100_log_entry.snap.SndLimTimeSnd) >= 9000000                             
  AND (web100_log_entry.snap.SndLimTimeRwin +                                   
    web100_log_entry.snap.SndLimTimeCwnd +                                      
    web100_log_entry.snap.SndLimTimeSnd) < 600000000                            
    
GROUP BY
  name, period, ts, download_mbps
""")

In [21]:
if False:
    df_ndt_dist = run_query("""
  -- Weekly cohort - NDT tests before and after the flow-control configuration change.
SELECT
  REGEXP_EXTRACT(connection_spec.server_hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS name,
  CASE
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-18") THEN 'before-2w'
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-18") AND TIMESTAMP("2018-02-25") THEN 'before-1w'
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-04") AND TIMESTAMP("2018-03-11") THEN 'after-1w'
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-18") THEN 'after-2w'
    ELSE 'what'
  END AS period,
  8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd)) AS download_mbps
FROM
  `measurement-lab.base_tables.ndt*`
WHERE
  ( -- One week - Sunday to Saturday.
    TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-18")
    OR TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-18") AND TIMESTAMP("2018-02-25")
    OR TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-04") AND TIMESTAMP("2018-03-11")
    OR TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-18"))
  AND REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw02|lga03)")
  AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
  AND web100_log_entry.connection_spec.remote_ip IN( (
    SELECT
      remote_ip
    FROM (
      SELECT
        web100_log_entry.connection_spec.remote_ip AS remote_ip
      FROM
        `measurement-lab.base_tables.ndt*`
      WHERE
        REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw02|lga03)")
        AND TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11")
        AND TIMESTAMP("2018-02-18")
        AND connection_spec.data_direction = 1
      GROUP BY
        remote_ip )
    INNER JOIN (
      SELECT
        web100_log_entry.connection_spec.remote_ip AS remote_ip
      FROM
        `measurement-lab.base_tables.ndt*`
      WHERE
        REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw02|lga03)")
        AND TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-18")
        AND TIMESTAMP("2018-02-25")
        AND connection_spec.data_direction = 1
      GROUP BY
        remote_ip )
    USING
      (remote_ip)
    INNER JOIN (
      SELECT
        web100_log_entry.connection_spec.remote_ip AS remote_ip
      FROM
        `measurement-lab.base_tables.ndt*`
      WHERE
        REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw02|lga03)")
        AND TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-04")
        AND TIMESTAMP("2018-03-11")
        AND connection_spec.data_direction = 1
      GROUP BY
        remote_ip )
    USING
      (remote_ip)
    INNER JOIN (
      SELECT
        web100_log_entry.connection_spec.remote_ip AS remote_ip
      FROM
        `measurement-lab.base_tables.ndt*`
      WHERE
        REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw02|lga03)")
        AND TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11")
        AND TIMESTAMP("2018-03-18")
        AND connection_spec.data_direction = 1
      GROUP BY
        remote_ip )
    USING
      (remote_ip)) )
GROUP BY
  name,
  period,
  download_mbps
""")

In [22]:
if False:
    df_ndt_dist = run_query("""
  -- Two-week cohorts - NDT tests before and after the flow-control configuration change.
SELECT
  REGEXP_EXTRACT(connection_spec.server_hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS name,
  CASE
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25") THEN 'before-2w'
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25") THEN 'after-2w'
    ELSE 'what'
  END AS period,
  --web100_log_entry.connection_spec.remote_ip AS remote_ip,
  (8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd))) AS download_mbps
  --MAX(8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd))) AS download_mbps
  --APPROX_QUANTILES(8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd)), 101)[ORDINAL(50)] as download_mbps
FROM
  `measurement-lab.base_tables.ndt*`
WHERE
  ( -- One week - Sunday to Saturday.
       TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25")
    OR TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25"))
  AND REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw|lga|nuq)\d\d")
  AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
  AND connection_spec.data_direction = 1
  AND web100_log_entry.connection_spec.remote_ip IN(
    SELECT
      remote_ip
    FROM (
      SELECT
        web100_log_entry.connection_spec.remote_ip AS remote_ip, count(*) as c1
      FROM
        `measurement-lab.base_tables.ndt*`
      WHERE
        REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw|lga|nuq)\d\d")
        AND TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25")
        AND connection_spec.data_direction = 1
      GROUP BY
        remote_ip
    ) INNER JOIN (
      SELECT
        web100_log_entry.connection_spec.remote_ip AS remote_ip, count(*) as c2
      FROM
        `measurement-lab.base_tables.ndt*`
      WHERE
        REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw|lga|nuq)\d\d")
        AND TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25")
        AND connection_spec.data_direction = 1
      GROUP BY
        remote_ip
    ) USING (remote_ip)) 
GROUP BY
  name,
  period,
  --remote_ip
  download_mbps
""")

In [23]:
df_ndt_dist = run_query("""
WITH mlab_ndt AS (
  SELECT
    connection_spec.server_hostname as server_hostname,
    log_time,
    web100_log_entry.connection_spec.remote_ip AS remote_ip,
    (8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd))) AS download_mbps
  FROM
    `measurement-lab.base_tables.ndt*`
  WHERE

  (    TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25")
    OR TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25"))
  AND REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw|lga|nuq)\d\d")
  AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
  AND connection_spec.data_direction = 1
  
  GROUP BY
    server_hostname,
    log_time,
    web100_log_entry.connection_spec.remote_ip,
    web100_log_entry.connection_spec.local_ip,
    web100_log_entry.connection_spec.remote_port,
    web100_log_entry.connection_spec.local_port,
    download_mbps)
    
SELECT
  REGEXP_EXTRACT(server_hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS name,
  CASE
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25") THEN 'before-2w'
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25") THEN 'after-2w'
    ELSE 'what'
  END AS period,
  --web100_log_entry.connection_spec.remote_ip AS remote_ip,
  --(8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd))) AS download_mbps
  MAX(download_mbps) AS download_mbps
  --APPROX_QUANTILES(download_mbps, 101)[ORDINAL(50)] as download_mbps

FROM
  mlab_ndt
WHERE
  remote_ip IN(
    SELECT
      remote_ip
    FROM (
      SELECT
        remote_ip, count(*) as c1
      FROM
        mlab_ndt
      WHERE
        TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25")
      GROUP BY
        remote_ip
      HAVING c1 > 10
    ) INNER JOIN (
      SELECT
        remote_ip AS remote_ip, count(*) as c2
      FROM
        mlab_ndt
      WHERE
        TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25")
      GROUP BY
        remote_ip
      HAVING c2 > 10
    ) USING (remote_ip)) 
GROUP BY
  name,
  period,
  remote_ip
  --download_mbps
""")

In [178]:
print df_ndt_dist.keys()
print len(df_ndt_dist)

def hist(vals, bin_count, log=True, cdf=False):
    """Produces hist or cdf values for smooth plots."""
    if log:
        r = [math.log10(x) for x in vals]
    else:
        r = vals
        
    m, bins = np.histogram(r, bin_count, normed=True)
    m = m.astype(float)

    tops = m
    if cdf:
        tops = np.cumsum(m)
        total = sum(m)
        tops = [float(t) / total for t in tops ]
    
    return tops, bins

seq = [(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]

for site in set([v[6:9] for v in set(df_ndt_dist['name'])]):
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 6))
    for p, h in enumerate(sorted([h for h in set(df_ndt_dist['name']) if site in h])):
        before = None
        r_before = None

        for day in ['before-2w', 'after-2w']:
            ds = df_ndt_dist[ (df_ndt_dist['name'] == h) & (df_ndt_dist['period'] == day) ]
            r = ds['download_mbps']
            print h, len(r)
            if not len(r):
                continue

            size = int(math.sqrt(len(r)))
            
            if day == 'after-2w':
                size = before
                # Test before vs after
                result = stats.ks_2samp(r, r_before)
                if result.pvalue < 0.01:
                    print 'diff', h, result
                
                # Test itself.
                a, b = train_test_split(r, test_size=0.5)
                result = stats.ks_2samp(a, b)
                if result.pvalue < 0.01:
                    print 'same', h, result
                    print '================================='

            else:
                before = size
                r_before = r
            
                
#            tops, bins = hist(r, int(1.8 * math.sqrt(len(r))), log=True , cdf=True)
            #tops, bins = hist(r, int(math.sqrt(len(r))), log=True , cdf=True)
            print size, h, day
            #tops, bins = hist(r, size, log=True , cdf=True)
            tops, bins = hist(r, size, log=True , cdf=True)
#            tops, bins = hist(r, int(1.8 * math.sqrt(len(r))), log=False , cdf=True)           
#            tops, bins = hist(r, len(r), log=False , cdf=True)            
            

            #tops_a, bins_a = hist(a, int(1 * math.sqrt(len(a))), log=True, cdf=True)
            #tops_b, bins_b = hist(b, int(1 * math.sqrt(len(b))), log=True, cdf=True)
            if p > len(seq)-1:
                print 'skipping', h
                continue
            i, j = seq[p]
#            print h, len(bins), len(tops)
            axes[i, j].plot(bins[:-1], tops, label='cdf-'+h[6:11] + '-' + str(day))
            #axes[i, j].plot(bins_a[:-1], tops_a, label=h[6:11] + '-' + str(day)+'-a')
            #axes[i, j].plot(bins_b[:-1], tops_b, label=h[6:11] + '-' + str(day)+'-b')
            axes[i, j].set_title(h[6:11])
            #axes[i, j].set_xlim(-10, 1000)
#            axes[i, j].set_xlim(math.log10(.25), math.log10(1000))
            axes[i, j].set_xlim(math.log10(.1), math.log10(1000))
            axes[i, j].grid(color='#dddddd')
            axes[i, j].legend(loc=2, fontsize='x-small')
            #axes[i, j].set_ylim(-0.1, 1.1)
            axes[i, j].xaxis.set_major_formatter(logFormatter)
    fig.subplots_adjust(hspace=0.3, wspace=0.4)
    fig.suptitle('NDT Download Distributions')

Index([u'download_mbps', u'name', u'period'], dtype='object')
44464
mlab1.dfw01 1465
38 mlab1.dfw01 before-2w
mlab1.dfw01 1447
38 mlab1.dfw01 after-2w
mlab1.dfw02 1433
37 mlab1.dfw02 before-2w
mlab1.dfw02 1438
37 mlab1.dfw02 after-2w
mlab1.dfw03 1431
37 mlab1.dfw03 before-2w
mlab1.dfw03 1434
37 mlab1.dfw03 after-2w
mlab1.dfw04 1428
37 mlab1.dfw04 before-2w
mlab1.dfw04 1440
37 mlab1.dfw04 after-2w
mlab1.dfw05 1836
42 mlab1.dfw05 before-2w
mlab1.dfw05 1855
42 mlab1.dfw05 after-2w
mlab1.dfw06 1434
37 mlab1.dfw06 before-2w
mlab1.dfw06 1439
37 mlab1.dfw06 after-2w
mlab1.lga02 525
22 mlab1.lga02 before-2w
mlab1.lga02 2153
diff mlab1.lga02 Ks_2sampResult(statistic=0.1738075332315927, pvalue=1.231555712986171e-11)
22 mlab1.lga02 after-2w
mlab1.lga03 1699
41 mlab1.lga03 before-2w
mlab1.lga03 1675
41 mlab1.lga03 after-2w
mlab1.lga04 1689
41 mlab1.lga04 before-2w
mlab1.lga04 1689
41 mlab1.lga04 after-2w
mlab1.lga05 2048
45 mlab1.lga05 before-2w
mlab1.lga05 2028
45 mlab1.lga05 after-2w
mlab1.lga06

In [25]:
#from scipy import stats

In [26]:
#len(n), len(bins[:-1])
#fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))

#axes.plot(bins[:-1], n)
#fig.subplots_adjust(hspace=0.3, wspace=0.4)
#print f


In [27]:
#m, bins = np.histogram(r, int(math.sqrt(len(ds['download_mbps']))))
#print m, bins, len(m), len(bins)

In [28]:
df_ndt_variance = run_query("""
WITH mlab_ndt AS (
  SELECT
    connection_spec.server_hostname as server_hostname,
    log_time,
    web100_log_entry.connection_spec.remote_ip AS remote_ip,
    (8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd))) AS download_mbps
  FROM
    `measurement-lab.base_tables.ndt*`
  WHERE

  (    TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25")
    OR TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25"))
  AND REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw|lga|nuq)\d\d")
  AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
  AND connection_spec.data_direction = 1
  
  GROUP BY
    server_hostname,
    log_time,
    web100_log_entry.connection_spec.remote_ip,
    web100_log_entry.connection_spec.local_ip,
    web100_log_entry.connection_spec.remote_port,
    web100_log_entry.connection_spec.local_port,
    download_mbps)


SELECT
  REGEXP_EXTRACT(server_hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS hostname,
  CASE
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25") THEN 'before-2w'
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25") THEN 'after-2w'
    ELSE 'what'
  END AS period,
  remote_ip,
  STDDEV(download_mbps) AS download_stddev

FROM
  mlab_ndt
WHERE
  remote_ip IN(
    SELECT
      remote_ip
    FROM (
      SELECT
        remote_ip, count(*) as c1
      FROM
        mlab_ndt
      WHERE
        TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25")
      GROUP BY
        remote_ip
      HAVING c1 > 5
    ) INNER JOIN (
      SELECT
        remote_ip AS remote_ip, count(*) as c2
      FROM
        mlab_ndt
      WHERE
        TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25")
      GROUP BY
        remote_ip
      HAVING c2 > 5
    ) USING (remote_ip)) 
GROUP BY
  hostname,
  period,
  remote_ip
  --download_mbps

HAVING download_stddev is not NULL
""")

In [29]:
print len(df_ndt_variance)

63423


In [179]:
seq = [(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]

skip = 'mlab1.lga02'
for site in set([v[6:9] for v in set(df_ndt_variance['hostname'])]):
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 6))
    for p, h in enumerate(sorted([h for h in set(df_ndt_variance['hostname']) if site in h])):
        before = None
        for day in ['before-2w', 'after-2w']:
            ds = df_ndt_variance[ (df_ndt_variance['hostname'] == h) & (df_ndt_variance['period'] == day) ]
            r = ds['download_stddev']
            print h, len(r)
            if not len(r) or h == skip:
                continue

            size = int(math.sqrt(len(r)))
            if day == 'after-2w':
                size = before
            else:
                before = size

            tops, bins = hist(r, size, log=True, cdf=False)
            if p > len(seq)-1:
                print 'skipping', h
                continue
            i, j = seq[p]

            axes[i, j].plot(bins[:-1], tops, label='cdf-'+h[6:11] + '-' + str(day))
    
            axes[i, j].set_title(h[6:11])
            #axes[i, j].set_xlim(-10, 1000)
            #axes[i, j].set_xlim(math.log10(.25), math.log10(1000))
            axes[i, j].set_xlim(math.log10(.01), math.log10(1000))
            axes[i, j].grid(color='#dddddd')
            axes[i, j].legend(loc=2, fontsize='x-small')
            #axes[i, j].set_ylim(-0.1, 1.1)
            axes[i, j].xaxis.set_major_formatter(logFormatter)
    fig.subplots_adjust(hspace=0.3, wspace=0.4)
    fig.suptitle('Distribution of Stddev of NDT Downloads per remote_ip')

mlab1.dfw01 1879
mlab1.dfw01 1906
mlab1.dfw02 2034
mlab1.dfw02 2008
mlab1.dfw03 1997
mlab1.dfw03 2056
mlab1.dfw04 1989
mlab1.dfw04 2037
mlab1.dfw05 2491
mlab1.dfw05 2518
mlab1.dfw06 2008
mlab1.dfw06 2065
mlab1.lga02 309
mlab1.lga02 2980
mlab1.lga03 2586
mlab1.lga03 2466
mlab1.lga04 2585
mlab1.lga04 2464
mlab1.lga05 3060
mlab1.lga05 2920
mlab1.lga06 2620
mlab1.lga06 2443
mlab1.lga07 2376
mlab1.lga07 2233
mlab1.nuq02 981
mlab1.nuq02 987
mlab1.nuq03 782
mlab1.nuq03 992
mlab1.nuq04 843
mlab1.nuq04 837
mlab1.nuq05 990
mlab1.nuq05 996
mlab1.nuq06 1014
mlab1.nuq06 971


In [180]:
print df_ndt_variance.keys()
print df_ndt_variance.axes
#print df_ndt_variance.query('')

Index([u'download_stddev', u'hostname', u'period', u'remote_ip'], dtype='object')
[RangeIndex(start=0, stop=63423, step=1), Index([u'download_stddev', u'hostname', u'period', u'remote_ip'], dtype='object')]


In [31]:
#print r

In [187]:
df_ndt_all = run_query("""
WITH mlab_ndt AS (
  SELECT
    REGEXP_EXTRACT(connection_spec.server_hostname, r"([a-z]{3})[0-9]{2}") as metro,
    REGEXP_EXTRACT(connection_spec.server_hostname, r"([a-z]{3}[0-9]{2})") as site,
    web100_log_entry.connection_spec.remote_ip as remote_ip,
    log_time,
    (8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd))) AS download_mbps

  FROM
  
    `measurement-lab.base_tables.ndt*`
  WHERE

        REGEXP_CONTAINS(connection_spec.server_hostname, r"(lga|dfw|nuq)\d\d")
    AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
    AND connection_spec.data_direction = 1
    AND web100_log_entry.connection_spec.remote_ip != "45.56.98.222"
    AND web100_log_entry.connection_spec.remote_ip != "2600:3c03::f03c:91ff:fe33:819"
    AND web100_log_entry.connection_spec.remote_ip != "35.225.75.192"
    AND web100_log_entry.connection_spec.remote_ip != "35.192.37.249"
    AND web100_log_entry.connection_spec.remote_ip != "35.193.254.117"
    AND anomalies.no_meta is not true
    
  
  GROUP BY
    connection_spec.server_hostname,
    log_time,
    web100_log_entry.connection_spec.remote_ip,
    web100_log_entry.connection_spec.local_ip,
    web100_log_entry.connection_spec.remote_port,
    web100_log_entry.connection_spec.local_port,
    download_mbps
)
    
SELECT
  metro,
  site,
  day,
 --  AVG(download_mbps) as download_mbps,
  APPROX_QUANTILES(download_mbps, 101)[ORDINAL(50)] as download_mbps,
  count(*) as count
FROM
(
  SELECT
    metro,
    site,
    TIMESTAMP_TRUNC(log_time, DAY) as day,
    -- APPROX_QUANTILES(download_mbps, 101)[ORDINAL(50)] as download_mbps
    MAX(download_mbps) as download_mbps
  FROM
    mlab_ndt

  GROUP BY
    metro, site, day, remote_ip
)

GROUP BY
  metro, site, day

ORDER BY
  day
""")

In [190]:
plot_scatter(
    df_ndt_all, 'day', 'download_mbps', axes_by='metro', group_by='site', axes=(3, 1),
    title='Median NDT Download Rates',
    ylabel="Mbps {0}",
    xlim=(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01")),
    ylim=(0, 50),
    fx=lambda l: [pd.to_datetime(t) for t in l],
    legend={'loc':2})

dfw
lga
nuq


In [189]:
#sites = [
#    ['dfw'],
#    ['lga'],
#    ['nuq'],
#]
#
#axes = [
#    [None],
#    [None],
#    [None],
#]
#print len(df_ndt_all)
#
#fig = plt.figure(figsize=(6, 8))
#
#for i, site_row in enumerate(sites):
#    for j, site in enumerate(site_row):
#        axes[i][j] = plt.subplot2grid((3, 1), (i, j))
#        axes[i][j].set_ylabel('Mbps ' + site.upper())
#        if i != len(sites)-1:
#            axes[i][j].set_xticklabels([])
#        
#        for s in set(df_ndt_all['site']):
#            if site in s:
#                ds = df_ndt_all[ (df_ndt_all['site'] == s) ]
#                d = [pd.to_datetime(t) for t in ds['day']]
#                axes[i][j].scatter(d, ds['download_mbps'], s=1, label=s)
#
#        #ds = df_ndt_all[ df_ndt_all['site'].str.contains(site) ]
#        #d = [pd.to_datetime(t) for t in ds['day']]
#        #axes[i][j].scatter(d, ds['download_mbps'], s=1, label='all')
#
#        #axes[i][j].set_ylim(100, 1000000)
#        axes[i][j].set_ylim(0, 50)
#        axes[i][j].set_xlim(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01"))
#        axes[i][j].tick_params(axis='x', labelrotation=-90)
#        axes[i][j].grid(color='#dddddd')
#        axes[i][j].legend(loc=2, fontsize='x-small')
#        #axes[i][j].semilogy()

#axes[1][0] = plt.subplot2grid((2, 1), (1, 0))
#axes[1][0].plot(df_ndt_all['day'], df_ndt_all['count'])
        
#fig.suptitle('Median NDT Download Rates')
#fig.tight_layout(rect=[0, 0.03, 1, 0.95])